<a href="https://colab.research.google.com/github/luisosmx/Git_exercises/blob/main/files_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()



In [2]:
import pandas as pd
from google.cloud import storage


In [3]:
project_id = 'calm-dragon-378604'
bucket_name = 'files_csv_lms'

client = storage.Client(project=project_id)
bucket = client.get_bucket(bucket_name)


In [4]:
file_name = 'movie_review2.xlsx'


In [32]:
blob = bucket.blob(file_name)
content = blob.download_as_string()
#Aqui se encuentra definido el dataframe = df
df = pd.read_excel(content)


In [48]:


df['review'] = df['review_str'].astype(str)
df['review'] = df['review_str'].replace('[', '')

In [49]:
df.head(10)

,cid,review_str,id_review,review
0,13756,[Once again Mr. Costner has dragged out a movi...,1,[Once again Mr. Costner has dragged out a movi...
1,15738,[This is an example of why the majority of act...,2,[This is an example of why the majority of act...
2,15727,"[First of all I hate those moronic rappers, w...",3,"[First of all I hate those moronic rappers, w..."
3,17954,[Not even the Beatles could write songs everyo...,4,[Not even the Beatles could write songs everyo...
4,16579,[Brass pictures (movies is not a fitting word ...,5,[Brass pictures (movies is not a fitting word ...
5,14841,[A funny thing happened to me while watching M...,6,[A funny thing happened to me while watching M...
6,18085,[This German horror film has to be one of the ...,7,[This German horror film has to be one of the ...
7,16365,"[Being a long-time fan of Japanese film, I ex...",8,"[Being a long-time fan of Japanese film, I ex..."
8,17912,[Tokyo Eyes tells of a 17 year old Japanese gi...,9,[Tokyo Eyes tells of a 17 year old Japanese gi...
9,15100,[Wealthy horse ranchers in Buenos Aires have a...,10,[Wealthy horse ranchers in Buenos Aires have a...


In [7]:
df.to_excel(file_name, index=False)
blob = bucket.blob(file_name)
blob.upload_from_filename(file_name)


In [8]:
blob = bucket.blob(file_name)
if blob.exists():
    print(f"El archivo {file_name} existe en el bucket {bucket_name}.")
else:
    print(f"El archivo {file_name} no existe en el bucket {bucket_name}.")


El archivo movie_review2.xlsx existe en el bucket files_csv_lms.


In [50]:
print(df.dtypes)


cid            int64
review_str    object
id_review      int64
review        object
dtype: object


In [10]:
from google.cloud import bigquery


In [14]:
client = bigquery.Client.from_service_account_json('/content/calm-dragon-378604-ae060da1908f.json')


In [29]:
table_id = 'calm-dragon-378604.exercise_gcp.movie_review'

schema_movie = [
    bigquery.SchemaField('cid', 'int64', mode='NULLABLE'),
    bigquery.SchemaField('review_str', 'object', mode='NULLABLE'),
    bigquery.SchemaField('id_review', 'int64', mode='NULLABLE')
]


In [31]:
job_config_movie = bigquery.LoadJobConfig(
  
    schema=schema_movie,
    
    write_disposition="WRITE_TRUNCATE",
)

In [34]:
job = client.load_table_from_dataframe(
      df, table_id, job_config=job_config_movie
  )  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
      "Loaded {} rows and {} columns to {}".format(
          table.num_rows, len(table.schema), table_id
      )
  )
 

/usr/local/lib/python3.8/dist-packages/google/cloud/bigquery/_pandas_helpers.py:266: UserWarning: Unable to determine type for field 'review_str'.
  warnings.warn("Unable to determine type for field '{}'.".format(bq_field.name))


BadRequest: ignored